In [2]:
import re
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [3]:
# Extract Time
def date_time(s):
    pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

# Find Authors or Contacts
def find_author(s):
    s = s.split(":")
    if len(s)==2:
        return True
    else:
        return False

# Finding Messages
def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message

In [4]:
data = []
conversation = 'chat.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [7]:
df = pd.DataFrame(data, columns=["Date", 'Time', 'Author', 'Message'])
df['Date'] = pd.to_datetime(df['Date'])
data = df.dropna()

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]

data.head(60)

,Date,Time,Author,Message,Positive,Negative,Neutral
0,2023-04-03,9:07 pm,Namrata,Hi,0.000,0.000,1.000
1,2023-04-03,9:07 pm,Namrata,How are you?,0.000,0.000,1.000
2,2023-04-03,9:36 pm,+91 9326334174,I'm good,0.744,0.000,0.256
3,2023-04-03,9:36 pm,+91 9326334174,And you?,0.000,0.000,1.000
4,2023-04-03,9:37 pm,Namrata,I'm also good,0.592,0.000,0.408
5,2023-04-03,9:42 pm,+91 9326334174,In which company you are in,0.000,0.000,1.000
6,2023-04-03,9:42 pm,Namrata,Goldstone technologies Hyderabad,0.000,0.000,1.000
7,2023-04-03,9:45 pm,+91 9326334174,Is it in IT field?,0.000,0.000,1.000
8,2023-04-03,9:45 pm,Namrata,Yes,1.000,0.000,0.000
9,2023-04-03,9:46 pm,+91 9326334174,Does it provide internship,0.000,0.000,1.000


In [25]:
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])

def sentiment_score(a, b, c):
    if (a>b) and (a>c):
        print("Positive 😊 ")
    elif (b>a) and (b>c):
        print("Negative 😠 ")
    else:
        print("Neutral 🙂 ")
sentiment_score(x, y, z)

Neutral 🙂 
